In [37]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
file_t = open('jester-data-1.csv').readlines()

In [3]:
mp = pd.read_csv('jester-data-1.csv', header=None).values

In [4]:
mp

array([[ 74.  ,  -7.82,   8.79, ...,  99.  ,  99.  ,  99.  ],
       [100.  ,   4.08,  -0.29, ...,   0.34,  -4.32,   1.07],
       [ 49.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       ...,
       [ 39.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 72.  ,   2.43,   2.67, ...,  99.  ,  99.  ,  99.  ]])

In [5]:
def get_matrix():
    return pd.read_csv('jester-data-1.csv', header=None, na_values=99).iloc[:,1:].values

In [72]:
class JokerObject:
    def __init__(self, matrix):
        self.matrix = matrix
        self.item_means = np.nanmean(matrix, 0)
        self.user_means = np.nanmean(matrix, 1)
    def get_matrix(self):
        return self.matrix
    def get_items_m(self):
        return self.item_means
    def get_item_m(self, key):
        return self.item_means[key]
    def get_users_m(self):
        return self.user_means
    def get_user_m(self, key):
        return self.user_means[key]
    def get_item(self, row, col):
        return self.matrix[row][col]
    def get_pearson_matrix(self):
        distances = np.zeros((len(self.user_means),len(self.user_means)))
        for i, user_1 in enumerate(self.matrix):
            for j, user_2 in enumerate(self.matrix):
                if(i>j):
                    continue
                if i != j:
                    distances[i,j] = np.sum((a-self.user_means[i])*(b-self.user_means[j]) for a,b in zip(user_1,user_2) if np.isnan(a)== False and np.isnan(b) == False)/np.sqrt(np.sum(np.square(a-self.user_means[i]) for index, a in enumerate(user_1) if np.isnan(a)== False and np.isnan(user_2[index]) == False)*np.sum(np.square(b-self.user_means[j]) for index, b in enumerate(user_2) if np.isnan(b)== False and np.isnan(user_1[index]) == False))
                    distances[j,i] = distances[i,j]
        return distances
                    

In [73]:
t = np.array([1,3,9])
t2 = np.array([5,6,2])
t3 = np.array([3,2,8])
t4 = np.array([t,t2,t3])

In [200]:
t4 = JokerObject(get_matrix()[0:5])

In [201]:
#m = JokerObject(t4)

In [202]:
# Got distance matrix
t0 = dt.datetime.now()
dist = t4.get_pearson_matrix()
print("time: ", dt.datetime.now() - t0)

time:  0:00:00.018651


In [203]:
def get_avg_weighted_sum(distance_matrix, data_matrix, coords):
    return data_matrix.get_user_m(coords[0]) + (1/distance_matrix.sum(axis=0)[coords[0]])*np.sum(distance_matrix[coords[0]][c_prime]*(data_matrix.get_item(c_prime, coords[1])-data_matrix.get_user_m(c_prime))for c_prime in range(distance_matrix.shape[0]) if not np.isnan(data_matrix.get_item(c_prime, coords[1])))

In [204]:
get_avg_weighted_sum(dist,t4,(0,0))

-0.9562849064966641

In [205]:
sum([1,2,3])

6

In [206]:
def get_average_ranking(data_matrix, coords):
    utilities = [data_matrix.get_item(c,coords[1])for c in range(data_matrix.get_matrix().shape[0]) if not np.isnan(data_matrix.get_item(c, coords[1]))]
    return (1/len(utilities))*sum(utilities)
    
    

In [285]:
get_average_ranking(t4, (0,53))

5.962000000000001

In [292]:
def get_adjusted_weighted_Nnn_sum(distance_matrix, data_matrix, coords, N = 3):
    user = coords[0]
    closest_users = dist[user,].argsort()[:N]
    return closest_users

In [293]:
get_adjusted_weighted_Nnn_sum(dist, t4, (0,1), 4)

array([0, 2, 1, 4])

In [264]:
tmp = dist[np.arange(dist.shape[0]) != 1][:,1]

array([-0.23975997, -0.07953065,  0.04503484, -0.11300586])

In [295]:
np.abs(tmp).argsort()

array([2, 1, 3, 0])

In [284]:
np.delete(dist[0,],0).argsort()[:3]

array([0, 1, 3])

In [250]:
t = np.delete(tmp,0)

In [254]:
t.argsort()[:3]

array([2, 0, 1])

In [182]:
dist[np.arange(dist.shape[0]) != 1]

array([[ 0.        , -0.23975997,  0.08422171,  0.39590991],
       [ 0.08422171, -0.07953065,  0.        ,  0.34000639],
       [ 0.39590991,  0.04503484,  0.34000639,  0.        ]])

In [213]:
dist

array([[ 0.        , -0.23975997,  0.08422171,  0.39590991,  0.33005565],
       [-0.23975997,  0.        , -0.07953065,  0.04503484, -0.11300586],
       [ 0.08422171, -0.07953065,  0.        ,  0.34000639,  0.33146463],
       [ 0.39590991,  0.04503484,  0.34000639,  0.        ,  0.44410961],
       [ 0.33005565, -0.11300586,  0.33146463,  0.44410961,  0.        ]])